In [24]:
%pip install matplotlib
%pip install tqdm
%pip install pandas

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn import preprocessing
from sklearn.decomposition import PCA
from scipy import stats
import numpy as np
import pandas as pd
import os

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [73]:
#Extract the 
features_data = pd.read_pickle("features.pkl")
#print(features_data[["label", "subject"]])

features_data_turncated = features_data.drop(columns=['label', 'subject'])

#features_data_turncated = features_data[["Mean_ecg", "mean_EDA", "WL_emg", "SSC_emg", "MAL_emg", "MCI_emg", "Mean_emg" "Median_emg", "STD_emg", "Mode_emg"]]
#features_data["Mean_ecg"] = preprocessing.normalize()

x = features_data_turncated.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
features_data_N = pd.DataFrame(x_scaled, index=features_data.index, columns=features_data_turncated.columns)

features_data_N = pd.concat([features_data_N, features_data[["label", "subject"]]], axis=1).reindex(features_data_N.index)

print(features_data_N)

     Mean_ecg  mean_EDA    WL_emg   SSC_emg   MAL_emg   MCI_emg      Mean_emg  \
0    0.957791  1.000000  0.441763  0.604681  1.000000  1.000000  0.000000e+00   
1    0.223410  0.814044  0.529353  0.509031  0.235217  0.228091  2.035409e-19   
2    0.399435  0.677774  0.307441  0.686339  0.438949  0.442457  3.614833e-19   
3    0.725063  0.571099  0.987081  0.684304  0.395546  0.430542  2.399701e-19   
4    0.294084  0.472124  0.585779  0.682015  0.395678  0.446363  3.500837e-19   
..        ...       ...       ...       ...       ...       ...           ...   
700  0.610476  0.039591  0.471503  0.796235  0.032958  0.000000  4.449153e-19   
701  0.490425  0.033640  0.604711  0.859323  0.030241  0.004876  2.402179e-19   
702  0.472677  0.020763  0.556437  0.245485  0.037132  0.040532  5.430511e-19   
703  0.477362  0.004003  0.492036  0.000000  0.016342  0.012736  3.064678e-19   
704  0.428008  0.000000  0.405875  0.014246  0.000000  0.002043  3.793262e-19   

     Median_emg   STD_emg  

In [122]:
pca = PCA(n_components=10)
pca_features = pca.fit_transform(features_data_N.drop(columns=['label', 'subject']))

print(pca_features)

[[ 1.24770449e+00  1.10655972e+00  1.36757994e-01 ... -1.31918496e-02
  -5.24183083e-16  5.41712167e-19]
 [ 1.04887413e-01 -9.91163908e-02 -4.14885580e-01 ...  1.97530498e-02
  -4.12912296e-16 -1.28816133e-18]
 [ 4.00401976e-01  2.26890167e-02 -1.88579465e-01 ...  4.26694757e-02
   6.79180999e-19  3.88521500e-18]
 ...
 [-7.76495435e-01  4.35088103e-02  7.50106087e-02 ... -1.68124625e-02
   1.14168102e-17  4.67567814e-18]
 [-9.29750863e-01  1.91297416e-01 -1.29024758e-02 ... -1.65120496e-02
   9.81018360e-18 -6.95603605e-19]
 [-9.48834180e-01  1.61084696e-01 -5.20197551e-02 ... -2.00776783e-02
   1.04061766e-17  9.41345215e-19]]


In [124]:
from sklearn.model_selection import train_test_split

#Split the data into a train and test set
X_train, x_test, Y_train, y_test = train_test_split(pca_features, features_data_N["label"], test_size=0.2,shuffle=True, random_state=102)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
x_test = scaler.transform(x_test)

Below the train data is fitted using the following classifiers: Random Forest Classifier, Random Forest Regressor, Decision Tree Regressor, Support Vector Machine, Neural Support Vector Machine, Linear Regression, Bernoulli

In [143]:
#Import classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import NuSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import BernoulliNB

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

#Fit data using classifiers and calculate accuracy
#Random Forest Classifier
classifier_RFC = RandomForestClassifier()
classifier_RFC.fit(X_train, Y_train)
y_pred_RFC = classifier_RFC.predict(x_test)
accuracy_RFC = accuracy_score(y_test, y_pred_RFC)
mse_RFC = mean_squared_error(y_test, y_pred_RFC)

#Decision Tree Regressor
classifier_DTC = DecisionTreeClassifier(max_depth=3)
classifier_DTC.fit(X_train, Y_train)
y_pred_DTC = classifier_DTC.predict(x_test)
accuracy_DTC = accuracy_score(y_test, y_pred_DTC)
mse_DTC = mean_squared_error(y_test, y_pred_DTC)

#Support Vector Machine
#classifier_SVM = NuSVC(kernel='rbf', nu=1)
#classifier_SVM.fit(X_train, Y_train)
#y_pred_SVM = classifier_SVM.predict(x_test)
#accuracy_SVM = accuracy_score(y_test, y_pred_SVM)
#mse_SVM = mean_squared_error(y_test, y_pred_SVM)

#Neural Support Vector Machine
#classifier_NSVM = NuSVC(kernel='linear')
#classifier_NSVM.fit(X_train, Y_train)
#y_pred_NSVM = classifier_NSVM.predict(x_test)
#accuracy_NSVM = accuracy_score(y_test, y_pred_NSVM)
#mse_NSVM = mean_squared_error(y_test, y_pred_NSVM)

#Linear Discriminant Analysis
classifier_LDA = LinearDiscriminantAnalysis()
classifier_LDA.fit(X_train, Y_train)
y_pred_LDA = classifier_LDA.predict(x_test)
accuracy_LDA = accuracy_score(y_test, y_pred_LDA)
mse_LDA = mean_squared_error(y_test, y_pred_LDA)

#Bernoulli
classifier_BNB = BernoulliNB()
classifier_BNB.fit(X_train, Y_train)
y_pred_BNB = classifier_BNB.predict(x_test)
accuracy_BNB = accuracy_score(y_test, y_pred_BNB)
mse_BNB = mean_squared_error(y_test, y_pred_BNB)

print("{} Accuracy, MSE: {:.2f}%, {:.2f}".format('Random Forrest Classifier', accuracy_RFC*100, mse_RFC))
print("{} Accuracy, MSE: {:.2f}%, {:.2f}".format('Decision Tree Regressor', accuracy_DTC*100, mse_DTC))
#print("{} Accuracy, MSE: {:.2f}%, {:.2f}".format('Support Vector Machine', accuracy_SVM*100, mse_SVM))
#print("{} Accuracy, MSE: {:.2f}%, {:.2f}".format('Neural Support Vector Machine', accuracy_NSVM*100, mse_NSVM))
print("{} Accuracy, MSE: {:.2f}%, {:.2f}".format('Linear Regression', accuracy_LDA*100, mse_LDA))
print("{} Accuracy, MSE: {:.2f}%, {:.2f}".format('Bernoulli', accuracy_BNB*100, mse_BNB))

Random Forrest Classifier Accuracy, MSE: 100.00%, 0.00
Decision Tree Regressor Accuracy, MSE: 90.07%, 0.10
Linear Regression Accuracy, MSE: 97.87%, 0.09
Bernoulli Accuracy, MSE: 77.30%, 0.58
